In [23]:
import pandas as pd

#taking the data and filtering for only Depression and Anxiety
sentiment = pd.read_csv('updated_sentiment_analysis_results.csv')
model_data= sentiment[sentiment['status'].isin(['Depression', 'Anxiety'])]
model_data

,Unnamed: 0,statement,status,cleaned_lemmatized_text,vader_sentiment
0,0,oh my gosh,Anxiety,oh my gosh,0.0000
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleep confuse mind restless heart all ...,-0.6908
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,all wrong back off dear forward doubt stay in ...,-0.7351
3,3,I've shifted my focus to something else but I'...,Anxiety,i 've shift my focus to something else but i '...,-0.5927
4,4,"I'm restless and restless, it's been a month n...",Anxiety,i 'm restless and restless it 's be a month no...,-0.4939
...,...,...,...,...,...
58187,53040,Is there any way to sleep better? I can't slee...,Anxiety,be there any way to sleep good i ca n't sleep ...,0.1635
58188,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety,public speak tip hi all i have to give a prese...,-0.6590
58189,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety,public speak tip hi all i have to give a prese...,-0.6590
58190,53042,I have really bad door anxiety! It's not about...,Anxiety,i have really bad door anxiety it 's not about...,-0.9195


In [24]:
# Extract the feature and target variables
X_text = model_data['cleaned_lemmatized_text']  # The text column
X_sentiment = model_data[['vader_sentiment']]   # The sentiment score column
y = model_data['status']                        # The target column (status)

# Check the shapes to ensure everything is correct
print(X_text.shape, X_sentiment.shape, y.shape)

(20559,) (20559, 1) (20559,)


so the tfidf_data .csv was good, but i needed it in a different shape so I redid it here so i could combine it with the other key pieces of information needed

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the text data into TF-IDF features
X_tfidf = vectorizer.fit_transform(X_text)

# Check the shape of the TF-IDF features
print(X_tfidf.shape)


(20559, 29321)


In [26]:
from scipy.sparse import hstack

# Combine TF-IDF features and sentiment features
X_combined = hstack([X_tfidf, X_sentiment])

# Check the shape of the combined feature matrix
print(X_combined.shape)

(20559, 29322)


In [27]:
from sklearn.model_selection import train_test_split

# Split the data into training and test sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Check the shape of the train and test sets
print(X_train.shape, X_test.shape)

(16447, 29322) (4112, 29322)


In [28]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Initialize the SVM classifier
svm_model = SVC(kernel='linear')  # You can experiment with other kernels like 'rbf'

# Train the model
svm_model.fit(X_train, y_train)

# Predict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.941147859922179
Classification Report:
               precision    recall  f1-score   support

     Anxiety       0.89      0.82      0.85       856
  Depression       0.95      0.97      0.96      3256

    accuracy                           0.94      4112
   macro avg       0.92      0.90      0.91      4112
weighted avg       0.94      0.94      0.94      4112



1. Accuracy - we classified correctly 94.1% of all the samples in our test set. Its a good performancew before we do any hyperparameter tuning, especially with just a binary classification problem.

2. Macro AVerages Precision (92%), Recall (90%), F1-Score(91%) is the mean of the metrics, slightly favoring depression due to larger recalla nd precision, and also  makes sense because the n amount of examples he model trained with depression was 3.2k while anxiety only had 850 samples.

3. Weighted AVerage 94% across the board, takes into account the support (number of samples in each class). Obviously more robust for depression, but still not bad for Anxiety.

Anxiety:
- precision score was 0.89 meaning its correct 89% and fewer false positives.
- recall score was 0.82 which means it correctly identified 82% of all actual anxiety instances, this means there were 18% false negatives
- F-1 Score was 0.85 whcih means its a harmonic measure of the two (balance between precision and recall)

Depression:
- Precision was 0.95 or 95% so the model is highly accurate in identifying depression
- Recall was 0.97 or 97% so it had very few false negatives
- F-1 Score was 0.96 wehich id very high blend of both the previous scores



In [30]:
model_data.to_csv('model_data.csv', index=False)
from google.colab import files

# Download the CSV file
files.download('model_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>